Chains in LangChain
- LLMChain
- Sequential Chains -> SimpleSequentialChain and SequentialChain
- Router Chain

In [12]:
%pip install python-dotenv
%pip install openai
%pip install pandas
#Install langchain
%pip install --upgrade langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.5 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 9.4 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 KB 6.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file)

In [3]:
#Load openai key
#link to get openai API key - https://platform.openai.com/account/api-keys
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [10]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [35]:
df.head()

,Product,Review
0,Queen Size Sheet Set,It was good
1,Waterproof Phone Pouch,Loved the waterproof sac
2,Luxury Air mattresses,Mattresses has a small hole at the top
3,Pillows insert,Best throw pillow on the Amazon
4,Milk Frother Handheld,Loved the product


LLM Chain

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [14]:
llm = ChatOpenAI(temperature=0.9)

In [15]:
prompt = ChatPromptTemplate.from_template("What is the best name to describe a company \
                                          that makes {product}")

In [17]:
chain = LLMChain(llm=llm, prompt=prompt)

In [18]:
product = "Luxury Air mattresses"
chain.run(product)

'Opulence Air Rest'


## Simple Sequential Chain

In [19]:
from langchain.chains import SimpleSequentialChain

In [20]:
llm = ChatOpenAI(temperature=0.9)

#prompt template 1
first_prompt = ChatPromptTemplate.from_template("What is the best name to describe\
                                                a company that makes {product}?")

#Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [21]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template("Write a 20 words description for the following company: {company_name}")

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [22]:
overall_seq_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [23]:
overall_seq_chain.run(product)



> Entering new SimpleSequentialChain chain...
OpulenceRest
OpulenceRest is a luxurious and opulent company that provides top-quality, lavish rest and relaxation products and services.

> Finished chain.


'OpulenceRest is a luxurious and opulent company that provides top-quality, lavish rest and relaxation products and services.'

## Sequential Chain

In [26]:
from langchain.chains import SequentialChain

In [27]:
llm = ChatOpenAI(temperature=0.9)

#prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template("Translate the following review to english: \
                                                \n {Review}")
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [28]:
second_prompt = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence \
                                                 \n {English_Review}")
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [29]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [30]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [31]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [41]:
review = df[" Review"].iloc[2]
overall_chain(review)



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-h3fwZpVqdQt2FuGWKNbEWbDI on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-h3fwZpVqdQt2FuGWKNbEWbDI on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


> Finished chain.


{'Review': ' Mattresses has a small hole at the top',
 'English_Review': 'The mattress has a small hole at the top.',
 'summary': 'The review states that there is a small hole at the top of the mattress.',
 'followup_message': 'Thank you for taking the time to provide feedback on our mattress. We appreciate your honesty and apologize for any inconvenience caused by the small hole at the top. We strive to maintain the highest quality standards, and we understand the importance of delivering a flawless product to our customers. Rest assured, we take this matter seriously and will investigate the issue further. Please allow us to assist you in resolving this matter to your satisfaction. Should you require any additional information or assistance, please do not hesitate to reach out to our customer service team. We value your feedback and look forward to rectifying this situation.'}

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""